In [15]:
import os 
os.environ["TFF_CPP_MIN_LOG_LEVEL"]="2"

In [16]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10

In [17]:
# #for gpu
# physical_devices = tf.config.list_physical_devices("GPU")
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [18]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

In [19]:
print(x_train.shape)
print(y_train.shape)

(50000, 32, 32, 3)
(50000, 1)


## Sequential Api


In [28]:
s_model = keras.Sequential(
    [
        keras.Input(shape=(32, 32, 3)),
        layers.Conv2D(32, 3, padding="same", activation="relu"),
        layers.Conv2D(64, 3, padding="same",activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(128, 3, padding="same",activation="relu"),
        layers.Conv2D(128, 3, padding="valid",activation="relu"),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dense(10),
    ]
)


**Note**
- padding= 'valid' means the spatial shape will change according to  kernal shape. and 'same' will keep unchange the shape.
- layers.Conv2D(32,3): first argument 32 is the number of kernals/filters and the 3 represents the filter/kernal size.

In [29]:
#model summary
print(s_model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_14 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 conv2d_15 (Conv2D)          (None, 32, 32, 64)        18496     
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 16, 16, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 16, 16, 128)       73856     
                                                                 
 conv2d_17 (Conv2D)          (None, 14, 14, 128)       147584    
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 7, 7, 128)         0         
 g2D)                                                 

In [30]:
#model configuration
s_model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(lr=3e-4),
    metrics=["accuracy"],
)

In [31]:
#fit model
s_model.fit(x_train,y_train,batch_size=64,epochs=10,verbose=2)

Epoch 1/10
782/782 - 197s - loss: 1.4096 - accuracy: 0.4895 - 197s/epoch - 252ms/step
Epoch 2/10
782/782 - 193s - loss: 0.9026 - accuracy: 0.6820 - 193s/epoch - 247ms/step
Epoch 3/10
782/782 - 218s - loss: 0.7285 - accuracy: 0.7460 - 218s/epoch - 279ms/step
Epoch 4/10
782/782 - 277s - loss: 0.6104 - accuracy: 0.7872 - 277s/epoch - 354ms/step
Epoch 5/10
782/782 - 287s - loss: 0.5273 - accuracy: 0.8169 - 287s/epoch - 367ms/step
Epoch 6/10
782/782 - 280s - loss: 0.4449 - accuracy: 0.8426 - 280s/epoch - 358ms/step
Epoch 7/10
782/782 - 292s - loss: 0.3698 - accuracy: 0.8701 - 292s/epoch - 374ms/step
Epoch 8/10
782/782 - 265s - loss: 0.3058 - accuracy: 0.8918 - 265s/epoch - 339ms/step
Epoch 9/10
782/782 - 252s - loss: 0.2571 - accuracy: 0.9093 - 252s/epoch - 322ms/step
Epoch 10/10
782/782 - 260s - loss: 0.2109 - accuracy: 0.9250 - 260s/epoch - 333ms/step


In [ ]:
#model evaluate
s_model.evaluate(x_test,y_test,batch_size=32,verbose=2)

313/313 - 2s - loss: 0.8116 - accuracy: 0.7321 - 2s/epoch - 6ms/step


[0.8115864992141724, 0.7321000099182129]

In [ ]:
print(s_model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 15, 15, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 flatten (Flatten)           (None, 2048)              0

## functional API

In [ ]:
## functional model

inputs = keras.Input(shape=(32, 32, 3))
x = layers.Conv2D(32, 3)(inputs)
x = layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(64, 3)(x)
x = layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(128, 3)(x)
x = layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = layers.Flatten()(x)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(10)(x)
f_model = keras.Model(inputs=inputs, outputs=outputs)

**Note**
- if we use batch normalization then after normalization we use activation function
- MaxPooling2D(): empty argument represents pool_size as 2,2

In [ ]:
# model configuration
f_model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(lr=3e-4),
    metrics=["accuracy"]
)

In [ ]:
# model fit
f_model.fit(x_train,y_train,batch_size=64,epochs=10,verbose=2)

Epoch 1/10
782/782 - 57s - loss: 1.2342 - accuracy: 0.5601 - 57s/epoch - 73ms/step
Epoch 2/10
782/782 - 55s - loss: 0.8799 - accuracy: 0.6923 - 55s/epoch - 70ms/step
Epoch 3/10
782/782 - 53s - loss: 0.7358 - accuracy: 0.7443 - 53s/epoch - 68ms/step
Epoch 4/10
782/782 - 55s - loss: 0.6376 - accuracy: 0.7781 - 55s/epoch - 70ms/step
Epoch 5/10
782/782 - 54s - loss: 0.5623 - accuracy: 0.8028 - 54s/epoch - 70ms/step
Epoch 6/10
782/782 - 54s - loss: 0.4924 - accuracy: 0.8277 - 54s/epoch - 69ms/step
Epoch 7/10
782/782 - 55s - loss: 0.4286 - accuracy: 0.8499 - 55s/epoch - 70ms/step
Epoch 8/10
782/782 - 55s - loss: 0.3707 - accuracy: 0.8712 - 55s/epoch - 70ms/step
Epoch 9/10
782/782 - 55s - loss: 0.3217 - accuracy: 0.8856 - 55s/epoch - 70ms/step
Epoch 10/10
782/782 - 54s - loss: 0.2762 - accuracy: 0.9039 - 54s/epoch - 69ms/step


In [ ]:
#evaluate model
f_model.evaluate(x_test,y_test,batch_size=64,verbose=2)

157/157 - 3s - loss: 1.0496 - accuracy: 0.7036 - 3s/epoch - 19ms/step


[1.049630880355835, 0.7035999894142151]

**Note** in the functional model the train accuracy is about 90 whereas the test aacuracy is 68. Thererfore here overfitting happens. We will work on this later. to overcome overfitting we will use regularization in the next section./

### SUGGESTION
- try tweaking the kernal size, training time e.t.c hyperparameters
